In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import numpy as np

In [2]:
grouped = pd.read_csv('../tensorMaker/total.csv')
meta = pd.read_csv("../tensorMaker/nftMetaData.csv")

In [3]:
grouped['timeStamp'] = pd.to_datetime(grouped['timeStamp'], utc=True)
grouped["timeStamp-day"] = grouped['timeStamp'].dt.floor('D')
grouped["nft_address"] = grouped["nft_address"].str.lower()

In [4]:
from scipy.stats import kurtosis
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import numpy as np
import functools

totalTensor = []

begin_of_window = grouped["timeStamp-day"].min()
window = 10
endOfWindow = np.datetime64(begin_of_window) + np.timedelta64(window,'D')

def disjunction(*conditions):
    return functools.reduce(np.logical_and, conditions)

while endOfWindow < np.datetime64(grouped["timeStamp-day"].max()):

    c_1 = grouped["timeStamp-day"] > pd.to_datetime(begin_of_window, utc=True)
    c_2 = grouped["timeStamp-day"] < pd.to_datetime(endOfWindow, utc=True)

    subSet = grouped[disjunction(c_1,c_2)]

    pivot_trades = pd.pivot_table(grouped, values="quant", index="timeStamp-day", columns="nft_address", aggfunc=np.sum)
    pivot_mean_log = pd.pivot_table(grouped, values="priceInDollarLog", index="timeStamp-day", columns="nft_address", aggfunc=np.mean)
    pivot_max_log = pd.pivot_table(grouped, values="priceInDollarLog", index="timeStamp-day", columns="nft_address", aggfunc=np.max)
    pivot_change_in_trades = pd.pivot_table(grouped, values="quant", index="timeStamp-day", columns="nft_address", aggfunc=np.sum).rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
    pivot_change_in_log_prices = pd.pivot_table(grouped, values="priceInDollarLog", index="timeStamp-day", columns="nft_address", aggfunc=np.mean).rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
    pivot_kurt_log = pd.pivot_table(grouped, values="priceInDollarLog", index="timeStamp-day", columns="nft_address", aggfunc=kurtosis)
    tensor = np.array([
        pivot_trades,
        pivot_max_log,
        pivot_change_in_trades,
        pivot_change_in_log_prices,
        pivot_kurt_log,
    ])

    totalTensor.append(tensor)
    print(len(totalTensor))
    begin_of_window = np.datetime64(begin_of_window) + np.timedelta64(window,'D')
    endOfWindow = np.datetime64(endOfWindow) + np.timedelta64(window,'D')

<ipython-input-4-aff6aa00d3aa>:12: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  endOfWindow = np.datetime64(begin_of_window) + np.timedelta64(window,'D')
<ipython-input-4-aff6aa00d3aa>:17: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  while endOfWindow < np.datetime64(grouped["timeStamp-day"].max()):


KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import kurtosis
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

kmeansResults = []

totalTensor = np.array(totalTensor)

for tensorI in range(totalTensor.shape[0]):
    tensor = totalTensor[tensorI]
    clusterDataList = []

    for series in range(tensor.shape[2]):
        clusterData = {}
        
        data = tensor[:,:, series]

        clusterData['nft']=pivot_trades.iloc[:,series].name
        clusterData['Length']=len(data[0,:][~np.isnan(data[0,:])])

        clusterData['changeSquared_trade']=np.mean(np.power(data[2,:][~np.isnan(data[2,:])],2))
        
        clusterData['changeSquared_price']=np.mean(np.power(data[3,:][~np.isnan(data[3,:])],2))
        clusterData['changeCube_price']=np.mean(np.power(data[3,:][~np.isnan(data[3,:])],3))

        for statisticsSeries in range(tensor.shape[0]):
            try:
                clusterData[statisticsSeries,'average'] =np.average(data[statisticsSeries,:][~np.isnan(data[statisticsSeries,:])])
                clusterData[statisticsSeries,'median'] =np.nanmedian(data[statisticsSeries,:])
                clusterData[statisticsSeries,'var'] =np.nanvar(data[statisticsSeries,:])
                clusterData[statisticsSeries,'mean'] = np.nanmean(data[statisticsSeries,:])
                clusterData[statisticsSeries,'std'] =np.nanstd(data[statisticsSeries,:])
                clusterData[statisticsSeries,'max'] =np.max(data[statisticsSeries,:][~np.isnan(data[statisticsSeries,:])])
                clusterData[statisticsSeries,'min'] =np.min(data[statisticsSeries,:][~np.isnan(data[statisticsSeries,:])])
                clusterData[statisticsSeries,'sum'] =np.sum(data[statisticsSeries,:][~np.isnan(data[statisticsSeries,:])])
                clusterData[statisticsSeries,'kurtosis'] = kurtosis(data[statisticsSeries,:][~np.isnan(data[statisticsSeries,:])])
                clusterDataList.append(clusterData)
            except Exception:
                continue

    x = pd.DataFrame(clusterDataList)

    X_total=  x.drop_duplicates()
    X_total = X_total.dropna(axis=0, how='any')
    pd.merge(X_total, meta, on="nft")

    X = X_total.iloc[:,1:]
    X_std = StandardScaler().fit_transform(X)

    pca = PCA()
    principalComponents = pca.fit_transform(X_std)
    features = range(pca.n_components_)
    PCA_components = pd.DataFrame(principalComponents)

    tSNE = TSNE(n_components=3)
    tSNE_COMPS = tSNE.fit_transform(PCA_components)
    tSNE_PCA_components = pd.DataFrame(tSNE_COMPS)
    
    kmeanstSNE = KMeans(n_clusters=3)
    labels = kmeanstSNE.fit_predict(tSNE_PCA_components)

    plt.scatter(tSNE_PCA_components[0], tSNE_PCA_components[1], c=labels, alpha=.2)
    # plt.savefig(str(tensorI)+".png")

    X_total["labels-tSNE_PCA"] = labels
    
    df = pd.DataFrame(X_total)
    df.to_csv(str(tensorI)+".csv")

    kmeansResults.append([dict(zip(X_total.iloc[:,1], labels))])

In [ ]:
import numpy as np
import sklearn.metrics as skMetrics
res = pd.DataFrame(kmeansResults)
comparison = res.shape[0]
comp_square = np.zeros((comparison,comparison))

for x in range(comparison):
    for y in range(comparison):
        comp_square[x][y] = skMetrics.adjusted_rand_score(res.iloc[x], res.iloc[y])

np.savetxt("foo.csv", comp_square, delimiter=",")